In [ ]:
#conda install -c conda-forge spacy

In [ ]:
#conda install nltk

# Import and subset data

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import re
import string
import spacy 
nlp = spacy.load('en_core_web_sm')
import nltk
from nltk.stem.snowball import SnowballStemmer
import gensim 
from gensim.models import Word2Vec
from time import time
from sklearn.model_selection import train_test_split

In [9]:
mydata = pd.read_csv('C:/Users/Janin/Downloads/CKME136/Full-Economic-News-DFE.csv', encoding = "ISO-8859-1")

In [10]:
mydata.head()

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,positivity,positivity:confidence,relevance,relevance:confidence,articleid,date,headline,positivity_gold,relevance_gold,text
0,842613455,False,finalized,3,12/5/15 17:48,3.0,0.6400,yes,0.640,wsj_398217788,8/14/91,Yields on CDs Fell in the Latest Week,NaN,NaN,NEW YORK -- Yields on most certificates of dep...
1,842613456,False,finalized,3,12/5/15 16:54,NaN,NaN,no,1.000,wsj_399019502,8/21/07,The Morning Brief: White House Seeks to Limit ...,NaN,NaN,The Wall Street Journal Online</br></br>The Mo...
2,842613457,False,finalized,3,12/5/15 1:59,NaN,NaN,no,1.000,wsj_398284048,11/14/91,Banking Bill Negotiators Set Compromise --- Pl...,NaN,NaN,WASHINGTON -- In an effort to achieve banking ...
3,842613458,False,finalized,3,12/5/15 2:19,NaN,0.0000,no,0.675,wsj_397959018,6/16/86,Manager's Journal: Sniffing Out Drug Abusers I...,NaN,NaN,The statistics on the enormous costs of employ...
4,842613459,False,finalized,3,12/5/15 17:48,3.0,0.3257,yes,0.640,wsj_398838054,10/4/02,Currency Trading: Dollar Remains in Tight Rang...,NaN,NaN,NEW YORK -- Indecision marked the dollar's ton...


In [11]:
data = mydata[['_unit_id','positivity', 'relevance', 'date','headline', 'text']]

data.head()

,_unit_id,positivity,relevance,date,headline,text
0,842613455,3.0,yes,8/14/91,Yields on CDs Fell in the Latest Week,NEW YORK -- Yields on most certificates of dep...
1,842613456,NaN,no,8/21/07,The Morning Brief: White House Seeks to Limit ...,The Wall Street Journal Online</br></br>The Mo...
2,842613457,NaN,no,11/14/91,Banking Bill Negotiators Set Compromise --- Pl...,WASHINGTON -- In an effort to achieve banking ...
3,842613458,NaN,no,6/16/86,Manager's Journal: Sniffing Out Drug Abusers I...,The statistics on the enormous costs of employ...
4,842613459,3.0,yes,10/4/02,Currency Trading: Dollar Remains in Tight Rang...,NEW YORK -- Indecision marked the dollar's ton...


# Exploratory Data Analysis

In [14]:
data['positivity'].fillna(0, inplace = True)
data.head(5)

C:\Users\Janin\Anaconda3\lib\site-packages\pandas\core\generic.py:6245: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


,_unit_id,positivity,relevance,date,headline,text
0,842613455,3.0,yes,8/14/91,Yields on CDs Fell in the Latest Week,NEW YORK -- Yields on most certificates of dep...
1,842613456,0.0,no,8/21/07,The Morning Brief: White House Seeks to Limit ...,The Wall Street Journal Online</br></br>The Mo...
2,842613457,0.0,no,11/14/91,Banking Bill Negotiators Set Compromise --- Pl...,WASHINGTON -- In an effort to achieve banking ...
3,842613458,0.0,no,6/16/86,Manager's Journal: Sniffing Out Drug Abusers I...,The statistics on the enormous costs of employ...
4,842613459,3.0,yes,10/4/02,Currency Trading: Dollar Remains in Tight Rang...,NEW YORK -- Indecision marked the dollar's ton...


In [15]:
data[['positivity','relevance']].groupby('relevance').count()

,positivity
relevance,
no,6571
not sure,9
yes,1420


No missing value in sentiment for relevant US economy news

In [18]:
data[['positivity','relevance']].groupby('positivity').count()

,relevance
positivity,
0.0,6580
2.0,35
3.0,343
4.0,255
5.0,205
6.0,214
7.0,295
8.0,71
9.0,2


# Pre-processing

Remove ambiguious data

In [19]:
data = data[data.relevance != 'not sure']

In [20]:
positivity = data['positivity'].value_counts()
positivity
# positivity range from 1 to 9, empty values represents that this article is not relevant to US economy

0.0    6571
3.0     343
7.0     295
4.0     255
6.0     214
5.0     205
8.0      71
2.0      35
9.0       2
Name: positivity, dtype: int64

In [21]:
# length of headline for relevant news
headline_len_r = data[data['relevance']=='yes']['headline'].str.len()
# number of words in headline for relevant news
headline_word_counts_r = data[data['relevance']=='yes']['headline'].str.split().str.len()


# length of headline for irrelevant news
headline_len_ir = data[data['relevance']=='no']['headline'].str.len()
# number of words in headline for irrelevant news
headline_word_counts_ir = data[data['relevance']=='no']['headline'].str.split().str.len()

print('Headline of relevant news has {} to {} characters, {} to {} words.'.format(headline_len_r.min(),
                                                                          headline_len_r.max(),
                                                                          headline_word_counts_r.min(),
                                                                          headline_word_counts_r.max()))
print('Headline of irrelevant news has {} to {} characters, {} to {} words.'.format(headline_len_ir.min(),
                                                                          headline_len_ir.max(),
                                                                          headline_word_counts_ir.min(),
                                                                          headline_word_counts_ir.max()))

Headline of relevant news has 6 to 251 characters, 1 to 38 words.
Headline of irrelevant news has 4 to 359 characters, 1 to 60 words.


In [22]:
headline_len_r.describe()

count    1420.000000
mean       66.289437
std        36.568721
min         6.000000
25%        38.000000
50%        61.000000
75%        89.000000
max       251.000000
Name: headline, dtype: float64

In [23]:
headline_word_counts_r.describe()

count    1420.000000
mean       10.791549
std         5.991908
min         1.000000
25%         6.000000
50%        10.000000
75%        14.000000
max        38.000000
Name: headline, dtype: float64

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(12, 10))
sns.distplot(headline_len_r, ax = axes[0,0])
axes[0, 0].set_title('length of headline for relevant news')
sns.distplot(headline_word_counts_r, ax = axes[0,1])
axes[0, 1].set_title('number of words in headline for relevant news')
sns.distplot(headline_len_ir, ax = axes[1,0])
axes[1, 0].set_title('length of headline for irrelevant news')
sns.distplot(headline_word_counts_ir, ax = axes[1,1])
axes[1, 1].set_title('number of words in headline for irrelevant news')


In [ ]:
# length of text for relevant news
text_len_r = data[data['relevance']=='yes']['text'].str.len()
# number of words in text for relevant news
text_word_counts_r = data[data['relevance']=='yes']['text'].str.split().str.len()


# length of text for irrelevant news
text_len_ir = data[data['relevance']=='no']['text'].str.len()
# number of words in text for irrelevant news
text_word_counts_ir = data[data['relevance']=='no']['text'].str.split().str.len()

print('Text of relevant news has {} to {} characters, {} to {} words.'.format(text_len_r.min(),
                                                                          text_len_r.max(),
                                                                          text_word_counts_r.min(),
                                                                          text_word_counts_r.max()))
print('Text of irrelevant news has {} to {} characters, {} to {} words.'.format(text_len_ir.min(),
                                                                          text_len_ir.max(),
                                                                          text_word_counts_ir.min(),
                                                                          text_word_counts_ir.max()))

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(12, 10))
sns.distplot(text_len_r, ax = axes[0,0])
axes[0, 0].set_title('length of text for relevant news')
sns.distplot(text_word_counts_r, ax = axes[0,1])
axes[0, 1].set_title('number of words in text for relevant news')
sns.distplot(text_len_ir, ax = axes[1,0])
axes[1, 0].set_title('length of text for irrelevant news')
sns.distplot(text_word_counts_ir, ax = axes[1,1])
axes[1, 1].set_title('number of words in text for irrelevant news')

## Normalization

Using String library

list of punctuation

In [24]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [25]:
def rm_punc(contents):
    no_punc = ''.join([x.lower() for x in contents if x not in string.punctuation])
    return no_punc

### Remove punctuation in headline

In [26]:
data['headline_new'] = data['headline'].apply(rm_punc)

### Remove punctuation in text 

In [27]:

data['text_new'] = data['text'].apply(rm_punc)


### Remove url and html in text 

In [28]:
def rm_url(contents):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'',contents)

def rm_html(contents):
    html=re.compile(r'<.*?>')
    return html.sub(r'',contents)

In [29]:
data['text_new'] = data['text_new'].apply(rm_url)
data['text_new'] = data['text_new'].apply(rm_html)

In [30]:
data.head(5)

,_unit_id,positivity,relevance,date,headline,text,headline_new,text_new
0,842613455,3.0,yes,8/14/91,Yields on CDs Fell in the Latest Week,NEW YORK -- Yields on most certificates of dep...,yields on cds fell in the latest week,new york yields on most certificates of depos...
1,842613456,0.0,no,8/21/07,The Morning Brief: White House Seeks to Limit ...,The Wall Street Journal Online</br></br>The Mo...,the morning brief white house seeks to limit c...,the wall street journal onlinebrbrthe morning ...
2,842613457,0.0,no,11/14/91,Banking Bill Negotiators Set Compromise --- Pl...,WASHINGTON -- In an effort to achieve banking ...,banking bill negotiators set compromise plan ...,washington in an effort to achieve banking re...
3,842613458,0.0,no,6/16/86,Manager's Journal: Sniffing Out Drug Abusers I...,The statistics on the enormous costs of employ...,managers journal sniffing out drug abusers is ...,the statistics on the enormous costs of employ...
4,842613459,3.0,yes,10/4/02,Currency Trading: Dollar Remains in Tight Rang...,NEW YORK -- Indecision marked the dollar's ton...,currency trading dollar remains in tight range...,new york indecision marked the dollars tone a...


## Tokenization

In [31]:
data['headline_new'] = [nlp(x) for x in data['headline_new']]
data['text_new'] = [nlp(x) for x in data['text_new']]

In [32]:
data.head(5)

,_unit_id,positivity,relevance,date,headline,text,headline_new,text_new
0,842613455,3.0,yes,8/14/91,Yields on CDs Fell in the Latest Week,NEW YORK -- Yields on most certificates of dep...,"(yields, on, cds, fell, in, the, latest, week)","(new, york, , yields, on, most, certificates,..."
1,842613456,0.0,no,8/21/07,The Morning Brief: White House Seeks to Limit ...,The Wall Street Journal Online</br></br>The Mo...,"(the, morning, brief, white, house, seeks, to,...","(the, wall, street, journal, onlinebrbrthe, mo..."
2,842613457,0.0,no,11/14/91,Banking Bill Negotiators Set Compromise --- Pl...,WASHINGTON -- In an effort to achieve banking ...,"(banking, bill, negotiators, set, compromise, ...","(washington, , in, an, effort, to, achieve, b..."
3,842613458,0.0,no,6/16/86,Manager's Journal: Sniffing Out Drug Abusers I...,The statistics on the enormous costs of employ...,"(managers, journal, sniffing, out, drug, abuse...","(the, statistics, on, the, enormous, costs, of..."
4,842613459,3.0,yes,10/4/02,Currency Trading: Dollar Remains in Tight Rang...,NEW YORK -- Indecision marked the dollar's ton...,"(currency, trading, dollar, remains, in, tight...","(new, york, , indecision, marked, the, dollar..."


## Lemmanization

In [33]:
data['headline_lemma'] = [[x.lemma_ for x in line] for line in data['headline_new']]
data['text_lemma'] = [[x.lemma_ for x in line] for line in data['text_new']]

In [34]:
data.head(5)

,_unit_id,positivity,relevance,date,headline,text,headline_new,text_new,headline_lemma,text_lemma
0,842613455,3.0,yes,8/14/91,Yields on CDs Fell in the Latest Week,NEW YORK -- Yields on most certificates of dep...,"(yields, on, cds, fell, in, the, latest, week)","(new, york, , yields, on, most, certificates,...","[yield, on, cd, fall, in, the, late, week]","[new, york, , yield, on, most, certificate, o..."
1,842613456,0.0,no,8/21/07,The Morning Brief: White House Seeks to Limit ...,The Wall Street Journal Online</br></br>The Mo...,"(the, morning, brief, white, house, seeks, to,...","(the, wall, street, journal, onlinebrbrthe, mo...","[the, morning, brief, white, house, seek, to, ...","[the, wall, street, journal, onlinebrbrthe, mo..."
2,842613457,0.0,no,11/14/91,Banking Bill Negotiators Set Compromise --- Pl...,WASHINGTON -- In an effort to achieve banking ...,"(banking, bill, negotiators, set, compromise, ...","(washington, , in, an, effort, to, achieve, b...","[banking, bill, negotiator, set, compromise, ...","[washington, , in, an, effort, to, achieve, b..."
3,842613458,0.0,no,6/16/86,Manager's Journal: Sniffing Out Drug Abusers I...,The statistics on the enormous costs of employ...,"(managers, journal, sniffing, out, drug, abuse...","(the, statistics, on, the, enormous, costs, of...","[managers, journal, sniff, out, drug, abuser, ...","[the, statistic, on, the, enormous, cost, of, ..."
4,842613459,3.0,yes,10/4/02,Currency Trading: Dollar Remains in Tight Rang...,NEW YORK -- Indecision marked the dollar's ton...,"(currency, trading, dollar, remains, in, tight...","(new, york, , indecision, marked, the, dollar...","[currency, trading, dollar, remain, in, tight,...","[new, york, , indecision, mark, the, dollar, ..."


## Stemming

In [35]:
stemmer = SnowballStemmer(language='english')

In [36]:
data['headline_new2'] = [[stemmer.stem(x) for x in line] for line in data['headline_lemma']]
data['text_new2'] = [[stemmer.stem(x) for x in line] for line in data['text_lemma']]

In [37]:
data.head(5)

,_unit_id,positivity,relevance,date,headline,text,headline_new,text_new,headline_lemma,text_lemma,headline_new2,text_new2
0,842613455,3.0,yes,8/14/91,Yields on CDs Fell in the Latest Week,NEW YORK -- Yields on most certificates of dep...,"(yields, on, cds, fell, in, the, latest, week)","(new, york, , yields, on, most, certificates,...","[yield, on, cd, fall, in, the, late, week]","[new, york, , yield, on, most, certificate, o...","[yield, on, cd, fall, in, the, late, week]","[new, york, , yield, on, most, certif, of, de..."
1,842613456,0.0,no,8/21/07,The Morning Brief: White House Seeks to Limit ...,The Wall Street Journal Online</br></br>The Mo...,"(the, morning, brief, white, house, seeks, to,...","(the, wall, street, journal, onlinebrbrthe, mo...","[the, morning, brief, white, house, seek, to, ...","[the, wall, street, journal, onlinebrbrthe, mo...","[the, morn, brief, white, hous, seek, to, limi...","[the, wall, street, journal, onlinebrbrth, mor..."
2,842613457,0.0,no,11/14/91,Banking Bill Negotiators Set Compromise --- Pl...,WASHINGTON -- In an effort to achieve banking ...,"(banking, bill, negotiators, set, compromise, ...","(washington, , in, an, effort, to, achieve, b...","[banking, bill, negotiator, set, compromise, ...","[washington, , in, an, effort, to, achieve, b...","[bank, bill, negoti, set, compromis, , plan, ...","[washington, , in, an, effort, to, achiev, ba..."
3,842613458,0.0,no,6/16/86,Manager's Journal: Sniffing Out Drug Abusers I...,The statistics on the enormous costs of employ...,"(managers, journal, sniffing, out, drug, abuse...","(the, statistics, on, the, enormous, costs, of...","[managers, journal, sniff, out, drug, abuser, ...","[the, statistic, on, the, enormous, cost, of, ...","[manag, journal, sniff, out, drug, abus, be, n...","[the, statist, on, the, enorm, cost, of, emplo..."
4,842613459,3.0,yes,10/4/02,Currency Trading: Dollar Remains in Tight Rang...,NEW YORK -- Indecision marked the dollar's ton...,"(currency, trading, dollar, remains, in, tight...","(new, york, , indecision, marked, the, dollar...","[currency, trading, dollar, remain, in, tight,...","[new, york, , indecision, mark, the, dollar, ...","[currenc, trade, dollar, remain, in, tight, ra...","[new, york, , indecis, mark, the, dollar, ton..."


## Stop-word Removal 

In [38]:
print(nlp.Defaults.stop_words)

{"'ll", 'those', 'cannot', 'this', 'nor', '‘s', 'but', 'seeming', 'bottom', 'n‘t', 'see', 'every', 'in', 'of', 'twelve', 'each', 'meanwhile', "'s", 'mostly', 'did', 'while', 'whereafter', "'d", 'on', 'all', 'already', 'without', 'thus', 'than', "'ve", 'fifteen', 'out', 'is', 'less', 'upon', 'or', 'beforehand', 'anything', 'me', 'its', 'very', 'more', 'one', "'re", 'hundred', 'now', 'thru', 'towards', 'several', 'again', 'are', 'during', 'therefore', 'being', 'under', 'herein', '‘re', 'had', 'third', 'afterwards', 'hereupon', 'a', 'three', 'call', 'hers', 'to', 'after', 'such', 'via', 'various', 'if', 'wherein', 'up', 'serious', 'say', 'does', 'be', 'among', 'least', 'first', 'him', 'though', 'few', 'throughout', 'who', 'please', 'nine', 'keep', 'everywhere', '’ll', 'her', 'once', 'side', 'even', 'rather', 'amongst', 'really', 'as', 'these', 'anyway', 'top', 'however', 'he', 'often', 'next', 'thereby', 'whence', 'eleven', 'an', 'must', 'down', 'the', 'move', 'herself', 'another', 'somet

In [39]:
def rm_stop(contents):
    no_stop = [x for x in contents if x not in nlp.Defaults.stop_words]
    return no_stop

In [40]:
data['headline_clean'] = data['headline_new2'].apply(rm_stop)
data['text_clean'] = data['text_new2'].apply(rm_stop)
data.head(5)

,_unit_id,positivity,relevance,date,headline,text,headline_new,text_new,headline_lemma,text_lemma,headline_new2,text_new2,headline_clean,text_clean
0,842613455,3.0,yes,8/14/91,Yields on CDs Fell in the Latest Week,NEW YORK -- Yields on most certificates of dep...,"(yields, on, cds, fell, in, the, latest, week)","(new, york, , yields, on, most, certificates,...","[yield, on, cd, fall, in, the, late, week]","[new, york, , yield, on, most, certificate, o...","[yield, on, cd, fall, in, the, late, week]","[new, york, , yield, on, most, certif, of, de...","[yield, cd, fall, late, week]","[new, york, , yield, certif, deposit, offer, ..."
1,842613456,0.0,no,8/21/07,The Morning Brief: White House Seeks to Limit ...,The Wall Street Journal Online</br></br>The Mo...,"(the, morning, brief, white, house, seeks, to,...","(the, wall, street, journal, onlinebrbrthe, mo...","[the, morning, brief, white, house, seek, to, ...","[the, wall, street, journal, onlinebrbrthe, mo...","[the, morn, brief, white, hous, seek, to, limi...","[the, wall, street, journal, onlinebrbrth, mor...","[morn, brief, white, hous, seek, limit, child,...","[wall, street, journal, onlinebrbrth, morn, br..."
2,842613457,0.0,no,11/14/91,Banking Bill Negotiators Set Compromise --- Pl...,WASHINGTON -- In an effort to achieve banking ...,"(banking, bill, negotiators, set, compromise, ...","(washington, , in, an, effort, to, achieve, b...","[banking, bill, negotiator, set, compromise, ...","[washington, , in, an, effort, to, achieve, b...","[bank, bill, negoti, set, compromis, , plan, ...","[washington, , in, an, effort, to, achiev, ba...","[bank, bill, negoti, set, compromis, , plan, ...","[washington, , effort, achiev, bank, reform, ..."
3,842613458,0.0,no,6/16/86,Manager's Journal: Sniffing Out Drug Abusers I...,The statistics on the enormous costs of employ...,"(managers, journal, sniffing, out, drug, abuse...","(the, statistics, on, the, enormous, costs, of...","[managers, journal, sniff, out, drug, abuser, ...","[the, statistic, on, the, enormous, cost, of, ...","[manag, journal, sniff, out, drug, abus, be, n...","[the, statist, on, the, enorm, cost, of, emplo...","[manag, journal, sniff, drug, abus, quick, fix]","[statist, enorm, cost, employe, drug, abus, kn..."
4,842613459,3.0,yes,10/4/02,Currency Trading: Dollar Remains in Tight Rang...,NEW YORK -- Indecision marked the dollar's ton...,"(currency, trading, dollar, remains, in, tight...","(new, york, , indecision, marked, the, dollar...","[currency, trading, dollar, remain, in, tight,...","[new, york, , indecision, mark, the, dollar, ...","[currenc, trade, dollar, remain, in, tight, ra...","[new, york, , indecis, mark, the, dollar, ton...","[currenc, trade, dollar, remain, tight, rang, ...","[new, york, , indecis, mark, dollar, tone, tr..."


## Pos Tagging 

In [41]:
data['headline_pos'] = [[x.pos_ for x in line] for line in data['headline_new']]
data['text_pos'] = [[x.pos_ for x in line] for line in data['headline_new']]
data.head(1)

,_unit_id,positivity,relevance,date,headline,text,headline_new,text_new,headline_lemma,text_lemma,headline_new2,text_new2,headline_clean,text_clean,headline_pos,text_pos
0,842613455,3.0,yes,8/14/91,Yields on CDs Fell in the Latest Week,NEW YORK -- Yields on most certificates of dep...,"(yields, on, cds, fell, in, the, latest, week)","(new, york, , yields, on, most, certificates,...","[yield, on, cd, fall, in, the, late, week]","[new, york, , yield, on, most, certificate, o...","[yield, on, cd, fall, in, the, late, week]","[new, york, , yield, on, most, certif, of, de...","[yield, cd, fall, late, week]","[new, york, , yield, certif, deposit, offer, ...","[NOUN, ADP, NOUN, VERB, ADP, DET, ADJ, NOUN]","[NOUN, ADP, NOUN, VERB, ADP, DET, ADJ, NOUN]"


In [42]:
data_clean = data[['positivity', 'relevance', 'date', 'headline_clean', 'text_clean', 'headline_pos', 'text_pos']]
data_clean.head(1)

,positivity,relevance,date,headline_clean,text_clean,headline_pos,text_pos
0,3.0,yes,8/14/91,"[yield, cd, fall, late, week]","[new, york, , yield, certif, deposit, offer, ...","[NOUN, ADP, NOUN, VERB, ADP, DET, ADJ, NOUN]","[NOUN, ADP, NOUN, VERB, ADP, DET, ADJ, NOUN]"


In [43]:
data_clean['headline_text'] = data_clean['headline_clean']+data_clean['text_clean']
data_clean['headline_text_pos'] =data_clean['headline_pos']+data_clean['text_pos']



C:\Users\Janin\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\Janin\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [44]:
data_clean.head(1)

,positivity,relevance,date,headline_clean,text_clean,headline_pos,text_pos,headline_text,headline_text_pos
0,3.0,yes,8/14/91,"[yield, cd, fall, late, week]","[new, york, , yield, certif, deposit, offer, ...","[NOUN, ADP, NOUN, VERB, ADP, DET, ADJ, NOUN]","[NOUN, ADP, NOUN, VERB, ADP, DET, ADJ, NOUN]","[yield, cd, fall, late, week, new, york, , yi...","[NOUN, ADP, NOUN, VERB, ADP, DET, ADJ, NOUN, N..."


## Data Split

In [46]:
data1 = data_clean[['relevance','headline_text','headline_text_pos']]
data2 = data_clean[['positivity','headline_text','headline_text_pos']][data_clean['relevance']=='yes']

In [48]:
data1.head(5)


,relevance,headline_text,headline_text_pos
0,yes,"[yield, cd, fall, late, week, new, york, , yi...","[NOUN, ADP, NOUN, VERB, ADP, DET, ADJ, NOUN, N..."
1,no,"[morn, brief, white, hous, seek, limit, child,...","[DET, NOUN, ADJ, PROPN, PROPN, VERB, PART, VER..."
2,no,"[bank, bill, negoti, set, compromis, , plan, ...","[NOUN, NOUN, NOUN, VERB, NOUN, SPACE, NOUN, PA..."
3,no,"[manag, journal, sniff, drug, abus, quick, fix...","[PROPN, PROPN, VERB, ADP, NOUN, NOUN, AUX, DET..."
4,yes,"[currenc, trade, dollar, remain, tight, rang, ...","[NOUN, NOUN, NOUN, VERB, ADP, ADJ, NOUN, ADP, ..."


In [49]:
data2.head(5)

,positivity,headline_text,headline_text_pos
0,3.0,"[yield, cd, fall, late, week, new, york, , yi...","[NOUN, ADP, NOUN, VERB, ADP, DET, ADJ, NOUN, N..."
4,3.0,"[currenc, trade, dollar, remain, tight, rang, ...","[NOUN, NOUN, NOUN, VERB, ADP, ADJ, NOUN, ADP, ..."
5,3.0,"[stock, fall, bofa, alcoa, slide, stock, decli...","[NOUN, VERB, ADV, PROPN, PROPN, PROPN, NOUN, V..."
9,4.0,"[dollar, fall, currenc, declin, soften, bond, ...","[PROPN, NOUN, VERB, ADP, ADJ, NOUN, NOUN, AUX,..."
12,4.0,"[defend, -pron-, deflat, author, jame, b, stew...","[VERB, PRON, ADP, NOUN, VERB, PRON, ADP, NOUN]"


In [51]:
text1 = data1['headline_text'].tolist()

In [52]:
text1 = data1['headline_text'].tolist()
text2 = data2['headline_text'].tolist()

pos1 = data1['headline_text_pos'].tolist()
pos2 = data2['headline_text_pos'].tolist()

relevance = data1['relevance'].tolist()
positivity = data2["positivity"].tolist()

In [57]:
text1_train, text1_test, pos_train, pos_test, relevance_train, relevance_test = train_test_split(text1, pos1, relevance, test_size=0.2, random_state=17)
text2_train, text2_test, pos_train, pos_test, positivity_train, positivity_test = train_test_split(text2, pos2, positivity, test_size=0.2, random_state=17)

In [64]:
train_size1 = len(text1_train)
test_size1 = len(text1_test)

train_size2 = len(text2_train)
test_size2 = len(text2_test)

In [67]:
print('data set 1 has {} instances in training set, {} instances in testing set.'.format(train_size1, test_size1))
print('data set 2 has {} instances in training set, {} instances in testing set.'.format(train_size2, test_size2))

data set 1 has 6392 instances in training set, 1599 instances in testing set.
data set 2 has 1136 instances in training set, 284 instances in testing set.


## Word Embedding 

In [70]:
word2vec = Word2Vec(data_clean['headline_text'], min_count=2)

In [71]:
vocabulary = word2vec.wv.vocab


In [72]:
print(word2vec.wv.most_similar(['yield']))

[('30year', 0.9073622226715088), ('10year', 0.9008407592773438), ('benchmark', 0.8838362693786621), ('threemonth', 0.8557829260826111), ('gilt', 0.8458844423294067), ('3132', 0.8428131937980652), ('twoyear', 0.842242956161499), ('treasurynot', 0.8356489539146423), ('bond\x89ûª', 0.8273123502731323), ('marketbrbrpric', 0.8253157138824463)]
